In [ ]:
%md 
### 1. Select the datasets

In [ ]:
%fs ls 'dbfs:/databricks-datasets/lending-club-loan-stats/'

In [ ]:
dbutils.fs.head("dbfs:/databricks-datasets/lending-club-loan-stats/lendingClubData.txt")

In [ ]:
%md 
### 2. Read the dataset

In [ ]:
lending_club_df = spark.read \
    .option("inferSchema", "true") \
    .option("header", "true") \
    .csv("dbfs:/databricks-datasets/lending-club-loan-stats/LoanStats_2018Q2.csv")

display(lending_club_df)


In [ ]:
lending_club_df.rdd.getNumPartitions()

In [ ]:
%md 
### 3. Create member_Id

In [ ]:
# -----use SHA-2 DSA FOR TO GENERATE UNIQUE MEMBER_ID
subset = lending_club_df['emp_title', 'emp_length', 'home_ownership', 'annual_inc', 'verification_status', 'grade', 'term', 'zip_code' ]

from pyspark.sql.functions import sha2, concat_ws

LCD_1 = lending_club_df.withColumn("member_id", sha2(concat_ws("||", * subset), 256))

In [ ]:
display(LCD_1)

In [ ]:
%md 
### 4. check the duplicate member_ids

In [ ]:
from pyspark.sql.functions import col
display(LCD_1.select("member_id").groupBy("member_id").count().alias("count").where(col("count") > 1).orderBy("count", ascending = False))

In [ ]:
%md
### 5. get rid of biased member_id records 
##### i.e., dublicate member_ids

In [ ]:
from pyspark.sql.functions import col

display(LCD_1.select("*").where(col("member_id") == "e9891d14a32ca8b2160b06c6d57ecff9cd66dcfa205fedab9ab9a4af1587a88e"))


In [ ]:
from pyspark.sql.functions import col

# Create a temporary DataFrame with member_id and their counts
LCD_TEMP = LCD_1.groupBy("member_id").count().withColumnRenamed("count", "member_count").where(col("member_count") > 1)

# Collect the member_ids with more than one occurrence into a list
member_ids_to_exclude = [row.member_id for row in LCD_TEMP.select("member_id").collect()]

# Filter the original DataFrame to exclude those member_ids
LCD_2 = LCD_1.filter(~col("member_id").isin(member_ids_to_exclude))

# Display the result
display(LCD_2)

#create temp table
LCD_2.createOrReplaceTempView("LCD_2")

In [ ]:
LCD_2.count()

In [ ]:
%md
### 6. Data Modelling

In [ ]:
# 1. borrower_info_df data
borrower_info_df = LCD_2.select(['member_id', 'emp_title', 'emp_length', 'home_ownership', 
                              'annual_inc', 'verification_status', 'zip_code', 'addr_state'])

In [ ]:
# 2. loan_details_df data
loan_details_df = LCD_2.select([ 'member_id', 'loan_amnt', 'funded_amnt', 'funded_amnt_inv', 
                             'term', 'int_rate', 'installment', 'grade', 'sub_grade', 'issue_d', 
                             'loan_status', 'pymnt_plan', 'purpose', 'title'])

In [ ]:
# 3. Payment and Recovery Dataset
payment_recovery_df = LCD_2.select(['member_id', 'total_pymnt', 'total_pymnt_inv', 
                                 'total_rec_prncp', 'total_rec_int', 'total_rec_late_fee', 
                                 'recoveries', 'collection_recovery_fee', 'last_pymnt_d', 
                                 'last_pymnt_amnt', 'next_pymnt_d'])

In [ ]:
# 4. Credit and Risk Dataset
credit_risk_df = LCD_2.select(['member_id', 'dti', 'delinq_2yrs', 'earliest_cr_line', 
                            'inq_last_6mths', 'mths_since_last_delinq', 'mths_since_last_record', 
                            'open_acc', 'pub_rec', 'revol_bal', 'revol_util', 'total_acc', 
                            'initial_list_status', 'out_prncp', 'out_prncp_inv'])

In [ ]:
%md
#### 7. Write back the segregated datasets into staging layer

In [ ]:
# write the dataframe into staging layer of dbfs
dbutils.fs.mkdirs("/lending_club_project/staging/")

borrower_info_df.repartition(1).write.format("parquet").mode("overwrite").option("header", "true").option("path", "/lending_club_project/staging/borrower_info/").save()

loan_details_df.repartition(1).write.format("parquet").mode("overwrite").option("header", "true").option("path", "/lending_club_project/staging/loan_details/").save()

payment_recovery_df.repartition(1).write.format("parquet").mode("overwrite").option("header", "true").option("path", "/lending_club_project/staging/payment_recovery/").save()

credit_risk_df.repartition(1).write.format("parquet").mode("overwrite").option("header", "true").option("path", "/lending_club_project/staging/credit_risk/").save()

In [ ]:
%md
#### 1.Cleaning of borrower_details_datasets

In [ ]:
%md 
#### 1a. define schema and assign column name

In [ ]:
columns = loan_info_df.columns
print(columns)
schema = 'member_id string, emp_title string , emp_length string, home_ownership string, annual_inc double, verification_status string, zip_code string, addr_state string'

In [ ]:
loan_info_df = spark.read.schema(schema).parquet("dbfs:/lending_club_project/staging/borrower_info")

In [ ]:
%md 
#### 1b. add ingestion date and country column default as USA

In [ ]:
from pyspark.sql.functions import *
dfa1 = loan_info_df.withColumn("ingestion_date", current_timestamp()).withColumn("country", lit("USA"))

In [ ]:
%md
#### 1C. remove records where annual_inc is zero
####### B/C they are not eligible for credit loan
####### dfa1.where(col("annual_inc") == 0).count()

In [ ]:
dfa2 = dfa1.where(col("annual_inc") > 0)

In [ ]:
%md 
#### 1d. convert 'null' employee title into others and convert the case into sentence case

In [ ]:
from pyspark.sql.functions import *
from pyspark.sql.types import StringType

# Define a Python function to convert text to sentence case
def to_sentence_case(text):
    if text:
        return text.title()
    return None

# Register the function as a UDF
to_sentence_case_udf = udf(to_sentence_case, StringType())

dfa2 = dfa2.withColumn("emp_title", to_sentence_case_udf(dfa2["emp_title"]))

dfa3 = dfa2.fillna({"emp_title": "Others"})

display(dfa3.select("emp_title").groupBy("emp_title").count().alias("count").orderBy("count", ascending = False))

In [ ]:
%md 
#### 1e.Convert employee length into integer and revert n/a into 0

In [ ]:
from pyspark.sql.functions import col, regexp_replace, avg, when
from pyspark.sql import SparkSession

# Assuming `spark` is your SparkSession and `dfa3` is your DataFrame

# Clean `emp_length` column using regexp_replace to keep only numeric characters
dfa3_cleaned = dfa3.withColumn("emp_length", regexp_replace(col("emp_length"), "[^0-9]", ""))

# Calculate average of emp_length (assuming it's numeric)
avg_emp_length = dfa3_cleaned.select(avg(col("emp_length"))).collect()[0][0]

# Replace blank cells with the calculated average and handle type casting
dfa4 = dfa3_cleaned.withColumn("emp_length", when(col("emp_length") == "", avg_emp_length).otherwise(col("emp_length"))) \
                           .withColumn("emp_length", col("emp_length").cast("int")) \
                           .withColumnRenamed("emp_length", "emp_length_in_years") \
                           .fillna({"emp_length_in_years": 6})  # Assuming you want to fill any remaining nulls

# Display the DataFrame
display(dfa4)


In [ ]:
%md
#### 1f. Verify any anomaly present in zip_code column and address_state column 

In [ ]:
from pyspark.sql.functions import length, col

# Assuming `df` is your DataFrame and `zip_code` is the column containing zip codes
zip_veri = dfa4.filter(length(col("zip_code")) > 5).count()

# Display or further process the filtered DataFrame
print(filtered_df)

# Assuming `df` is your DataFrame and `zip_code` is the column containing zip codes
addr_state_df = dfa4.filter(length(col("addr_state")) > 2).count()

# Display or further process the filtered DataFrame
print(addr_state_df)

In [ ]:
%md
#### 2. Cleaning of loan_details_datasets

In [ ]:
%md 
#### 2a. define schema and assign column name

In [ ]:
lend_info_df = spark.read.option("inferschema", "true").parquet("/lending_club_project/staging/loan_details/")
display(lend_info_df)

In [ ]:
%md
#### 2b. Convert term column into tenure_period_in_month and cast it into integer

In [ ]:
from pyspark.sql.functions import *
dfb1 = lend_info_df.withColumn("term", regexp_replace(col("term"), "[^0-9]", "")).withColumn("term", col("term").cast("int")).withColumnRenamed("term", "tenure_period_in_month")

In [ ]:
%md
#### 2c. Convert int_rate into interest_rate_in_percentage and cast it into double

In [ ]:
# Correct the code to remove non-numeric characters and cast the column to double
dfb2 = dfb1.withColumn("int_rate", regexp_replace(col("int_rate"), "[^0-9.]", "").cast("double")) \
           .withColumnRenamed("int_rate", "interest_rate_in_percentage")

In [ ]:
%md 
#### 2d. Convert issue_d into issued_date into date format

In [ ]:
from pyspark.sql.functions import date_format, concat, lit, to_date, col

# Convert 'issue_d' from string to date format
dfb3 = dfb2.withColumn("issue_d", to_date(col("issue_d"), "MMM-yy")).withColumnRenamed("issue_d","issued_date")

In [ ]:
%md
#### 2e. Cast purpose column into sentence case

In [ ]:
from pyspark.sql.functions import udf
from pyspark.sql.types import StringType

# Define a Python function to convert text to sentence case
def to_sentence_case(text):
    if text:
        return text.title()
    return None

# Register the UDF
to_sent_case_udf = udf(to_sentence_case, StringType())

# Apply the UDF to the 'purpose' column
dfb4 = dfb3.withColumn("purpose", to_sent_case_udf(col("purpose")))

In [ ]:
%md
#### 2f. Drop unnecessery Columns

In [ ]:
dfb5 = dfb4.drop("pymnt_plan", "title")
display(dfb5)

In [ ]:
%md
## 3. cleaning of payment_recovery_datasets

In [ ]:
%md 
#### 3a. define schema and assign column name

In [ ]:
payment_recovery_df = spark.read.option("inferSchema", "true").parquet("/lending_club_project/staging/payment_recovery/")
display(payment_recovery_df)

In [ ]:
%md 
#### Relationships:
I. total_pymnt = total_rec_prncp + total_rec_int + other_fees:

The total payments made (total_pymnt) are the sum of the principal repaid (total_rec_prncp), the interest paid (total_rec_int), and any other fees (e.g., late fees).

II. total_pymnt and total_pymnt_inv:
These should be closely related because the total payments made by the borrower (total_pymnt) should largely correspond to the payments received by the investors (total_pymnt_inv).
Differences may arise due to service fees deducted from the payments before they are passed on to investors.

III. total_rec_prncp and total_rec_int:

These two components together make up the bulk of total_pymnt.
Over time, as more of the loan principal is paid down, total_rec_prncp will increase.
The interest paid (total_rec_int) depends on the interest rate and the remaining principal balance over time.

In [ ]:
%md 
#### 3b. Cast last_pymnt_d in to date and rename col

In [ ]:
from pyspark.sql.functions import *
dfc1 = payment_recovery_df.withColumn("last_pymnt_d", to_date(col("last_pymnt_d"), "MMM-yy")).withColumnRenamed("last_pymnt_d", "last_payment_date")
dfc2 = dfc1.withColumn("next_pymnt_d", to_date(col("next_pymnt_d"), "MMM-yy")).withColumnRenamed("next_pymnt_d", "next_payment_date")
display(dfc2)

In [ ]:
%md
#### 4. Cleaning of Credit risk dataset

In [ ]:
%md 
#### 4a. define schema and assign column name

In [ ]:
credit_risk_df = spark.read.option("inferSchema", "true").parquet("/lending_club_project/staging/credit_risk/")
display(credit_risk_df)

In [ ]:
%md
#### Details to know
1. Personal and Loan Information: member_id uniquely identifies borrowers.
2. Financial Ratios: dti shows debt burden relative to income.
3. Delinquency Metrics: delinq_2yrs and mths_since_last_delinq track past due payments and recency.
4. Credit History: earliest_cr_line and inq_last_6mths show the length of credit history and recent inquiries.
5. Public Records: mths_since_last_record and pub_rec indicate public derogatory records.
6. Current Accounts: open_acc and total_acc show current and total credit accounts.
7. Revolving Credit: revol_bal and revol_util measure revolving credit balance and usage.
8. Loan Status: initial_list_status provides the initial status of loan listings.
9. Principal Balances: out_prncp and out_prncp_inv show outstanding principal balances from borrower and investor perspectives.

In [ ]:
%md 
#### 4a. Filling Null values

In [ ]:
from pyspark.sql.functions import col, trim, length, when, regexp_replace, to_date, mean, stddev

dfd1 = credit_risk_df.fillna({
    'dti': credit_risk_df.select(mean(col('dti'))).collect()[0][0],  # Fill with mean
    'delinq_2yrs': 0,  # Fill with 0 if no delinquency
    'inq_last_6mths': 0,
    'mths_since_last_delinq': -1,  # Placeholder for missing values
    'mths_since_last_record': -1,
    'pub_rec': 0
})

In [ ]:
%md
#### 4b. conversion of date earliest_cr_line

In [ ]:
# Data type conversion
dfd2 = dfd1.withColumn("earliest_cr_line", to_date(col("earliest_cr_line"), 'MM/yyyy'))

In [ ]:
%md
#### 4c. compare both mean and standard deveviation are in line or not
####### results found closer value data seems to be in line

In [ ]:
dti_mean = dfd2.select(mean(col('dti'))).collect()[0][0]
print(dti_mean)

In [ ]:
dti_stddev = dfd2.select(stddev(col('dti'))).collect()[0][0]
print(dti_stddev)

In [ ]:
%md
#### 4d. Drop duplicates

In [ ]:
# Removing duplicates
dfd3 = dfd2.dropDuplicates()

In [ ]:
%md 
#### 4e. cast revol_util into double

In [ ]:
dfd4 = dfd3.withColumn("revol_util", regexp_replace(col("revol_util"), "[^0-9.]", "")).withColumnRenamed("revol_util","revol_util_in_percentage").drop("mths_since_last_record").withColumn("revol_util_in_percentage", col("revol_util_in_percentage").cast("float"))
display(dfd4)

In [ ]:
%md
#### 5. ultra cleaned Credit risk dataset

In [ ]:
%md
#### 5a. exclude the missing value i.e -1 from mths_since_last_delinq

In [ ]:
dfd5 = dfd4.where(col("mths_since_last_delinq") != -1)
display(dfd5)

In [ ]:
display(dfd5.count())

In [ ]:
%md
#### 6. Write back datasets into delta tables

In [ ]:
dbutils.fs.mkdirs("dbfs:/lending_club_project/landing")

In [ ]:
dfa4.write.format("delta") \
    .mode("overwrite") \
    .option("path", "dbfs:/lending_club_project/landing/customers_details_datasets.delta") \
    .saveAsTable("customers_details_datasets")

dfb5.write.format("delta") \
    .mode("overwrite") \
    .option("path", "dbfs:/lending_club_project/landing/loan_details_datasets.delta") \
    .saveAsTable("loan_details_datasets")

dfc2.write.format("delta") \
    .mode("overwrite") \
    .option("path", "dbfs:/lending_club_project/landing/payment_recovery_datasets.delta") \
    .saveAsTable("payment_recovery_datasets")

dfd4.write.format("delta") \
    .mode("overwrite") \
    .option("path", "dbfs:/lending_club_project/landing/credit_risk_dataset.delta") \
    .saveAsTable("credit_risk_dataset")

dfd5.write.format("delta") \
    .mode("overwrite") \
    .option("path", "dbfs:/lending_club_project/landing/ultra_clean_credit_risk_dataset.delta") \
    .saveAsTable("ultra_clean_credit_risk_dataset")


In [ ]:
spark.conf.set("spark.databricks.delta.joinReorder.enabled", "true")

In [ ]:
%md
#### 7. Read delta tables

In [ ]:

lending_club_crunched_df = spark.sql("""SELECT 
    U.member_id,
    C.emp_title, 
    C.emp_length_in_years, 
    C.home_ownership, 
    C.annual_inc, 
    C.verification_status, 
    C.zip_code, 
    C.addr_state, 
    C.ingestion_date, 
    C.country,
    
    L.loan_amnt, 
    L.funded_amnt, 
    L.funded_amnt_inv, 
    L.tenure_period_in_month, 
    L.interest_rate_in_percentage, 
    L.installment, 
    L.grade, 
    L.sub_grade, 
    L.issued_date, 
    L.loan_status, 
    L.purpose,
    
    P.total_pymnt, 
    P.total_pymnt_inv, 
    P.total_rec_prncp, 
    P.total_rec_int, 
    P.total_rec_late_fee, 
    P.recoveries, 
    P.collection_recovery_fee, 
    P.last_payment_date, 
    P.last_pymnt_amnt, 
    P.next_payment_date,
    
    U.dti, 
    U.delinq_2yrs, 
    U.earliest_cr_line, 
    U.inq_last_6mths, 
    U.mths_since_last_delinq, 
    U.open_acc, 
    U.pub_rec, 
    U.revol_bal, 
    U.revol_util_in_percentage, 
    U.total_acc, 
    U.initial_list_status, 
    U.out_prncp, 
    U.out_prncp_inv

FROM ultra_clean_credit_risk_dataset U
LEFT JOIN customers_details_datasets C ON U.member_id = C.member_id
LEFT JOIN loan_details_datasets L ON U.member_id = L.member_id
LEFT JOIN payment_recovery_datasets P ON U.member_id = P.member_id""")

In [ ]:
# Finalised total cleaned records
display(lending_club_crunched_df.count())

In [ ]:

lending_club_crunched_df.write.format("delta") \
    .mode("overwrite") \
    .option("path", "dbfs:/lending_club_project/landing/lending_club_crunched_df.delta") \
    .saveAsTable("lending_club_crunched_df")

In [ ]:
%md
### BUSINESS USE CASE
##### 8. Calculating Credit Score

In [ ]:
%md
###### To calculate a credit score using a traditional approach without machine learning, you can use a scoring model based on weighted factors. Each factor is assigned a weight based on its importance in determining creditworthiness. Here's how you can do it:

1. Determine the Factors: Identify the key factors that will influence the credit score.
2. Assign Weights: Assign weights to each factor based on its importance.
3. Calculate Scores: Normalize and calculate the score for each factor.
4. Compute Final Credit Score: Aggregate the scores based on their weights.

Example Credit Score Calculation
Let's use the following factors:

Debt-to-Income Ratio (dti)
Number of Delinquencies in the Last 2 Years (delinq_2yrs)
Revolving Balance (revol_bal)
Revolving Utilization (revol_util_in_percentage)
Number of Open Accounts (open_acc)
Total Accounts (total_acc)
Outstanding Principal (out_prncp)
Outstanding Principal (Investor) (out_prncp_inv)

In [ ]:
DF1 = spark.read.format("delta").option("inferSchema", "true").load("dbfs:/lending_club_project/landing/lending_club_crunched_dataset.delta")
display(DF1)

In [ ]:
%md 
#### 8b.Drop rows with missing values in critical columns

In [ ]:
DF2 = DF1.dropna(subset=["member_id", "loan_amnt", "funded_amnt", "annual_inc", "dti", "delinq_2yrs", "revol_bal", "total_acc", "out_prncp", "out_prncp_inv"])

In [ ]:
%md
#### 8c.Filter out rows with revol_util_in_percentage > 100

In [ ]:
DF3 = DF2.filter(col("revol_util_in_percentage") <= 100)

In [ ]:
%md
#### 8d.Impute missing values in non-critical columns with the median or a placeholder value

In [ ]:
from pyspark.sql.functions import when
median_annual_inc = DF3.approxQuantile("annual_inc", [0.5], 0.0)[0]
DF4 = DF3.withColumn("annual_inc", when(col("annual_inc").isNull(), median_annual_inc).otherwise(col("annual_inc")))


In [ ]:
%md
#### 8e. Normalize and calculate scores for each factor

In [ ]:
DF4 = DF4.withColumn("income_score", when(col("annual_inc") > 100000, 100)
                                    .when(col("annual_inc") > 75000, 75)
                                    .when(col("annual_inc") > 50000, 50)
                                    .when(col("annual_inc") > 25000, 25)
                                    .otherwise(0))

DF4 = DF4.withColumn("dti_score", when(col("dti") < 10, 100)
                                  .when(col("dti") < 20, 75)
                                  .when(col("dti") < 30, 50)
                                  .otherwise(0))

DF4 = DF4.withColumn("delinq_score", when(col("delinq_2yrs") == 0, 100)
                                     .when(col("delinq_2yrs") == 1, 50)
                                     .otherwise(0))

DF4 = DF4.withColumn("revol_bal_score", when(col("revol_bal") < 5000, 100)
                                      .when(col("revol_bal") < 10000, 75)
                                      .when(col("revol_bal") < 20000, 50)
                                      .otherwise(0))

DF4 = DF4.withColumn("revol_util_score", when(col("revol_util_in_percentage") < 30, 100)
                                       .when(col("revol_util_in_percentage") < 50, 75)
                                       .when(col("revol_util_in_percentage") < 70, 50)
                                       .otherwise(0))

DF4 = DF4.withColumn("total_acc_score", when(col("total_acc") > 20, 100)
                                      .when(col("total_acc") > 10, 75)
                                      .when(col("total_acc") > 5, 50)
                                      .otherwise(0))

DF4 = DF4.withColumn("out_prncp_score", when(col("out_prncp") == 0, 100)
                                      .when(col("out_prncp") < 1000, 75)
                                      .when(col("out_prncp") < 5000, 50)
                                      .otherwise(0))

DF4 = DF4.withColumn("out_prncp_inv_score", when(col("out_prncp_inv") == 0, 100)
                                          .when(col("out_prncp_inv") < 1000, 75)
                                          .when(col("out_prncp_inv") < 5000, 50)
                                          .otherwise(0))

In [ ]:
%md
#### 8f.Aggregate Scores

In [ ]:
DF5 = DF4.withColumn("total_score", col("income_score") + col("dti_score") + col("delinq_score") + col("revol_bal_score") + col("revol_util_score") + col("total_acc_score") + col("out_prncp_score") + col("out_prncp_inv_score"))

In [ ]:
%md
#### 8g. Normalize the Score

In [ ]:
max_score = 800  # Maximum possible score based on our criteria
min_score = 300  # Minimum possible score to match typical credit score ranges
DF6 = DF5.withColumn("credit_score", (col("total_score") / max_score) * (850 - 300) + 300)


In [ ]:
%md
#### 8h. Grading the Credit Score

In [ ]:
DF7 = DF6.withColumn("credit_grade", when(col("credit_score") >= 750, "very_good")
                                    .when(col("credit_score") >= 700, "good")
                                    .when(col("credit_score") >= 650, "medium")
                                    .when(col("credit_score") >= 600, "bad")
                                    .otherwise("very_bad"))

In [ ]:
very_good_df = DF7.where(col("credit_grade")== 'very_good')
good_df = DF7.where(col("credit_grade")== 'good')
medium_df = DF7.where(col("credit_grade")== 'medium')
bad_df = DF7.where(col("credit_grade")== 'bad')
very_bad_df = DF7.where(col("credit_grade")== 'very_bad')

In [ ]:
%md
#### 9. write back the graded finalised results

In [ ]:
dbutils.fs.mkdirs("dbfs:/lending_club_project/reporting")

In [ ]:
very_good_df.write.format("delta") \
    .mode("overwrite") \
    .option("path", "dbfs:/lending_club_project/reporting/very_good_score_dataset.delta") \
    .saveAsTable("very_good_score_dataset")

good_df.write.format("delta") \
    .mode("overwrite") \
    .option("path", "dbfs:/lending_club_project/reporting/good_score_dataset.delta") \
    .saveAsTable("good_score_dataset")

medium_df.write.format("delta") \
    .mode("overwrite") \
    .option("path", "dbfs:/lending_club_project/reporting/medium_score_dataset.delta") \
    .saveAsTable("medium_score_dataset")

bad_df.write.format("delta") \
    .mode("overwrite") \
    .option("path", "dbfs:/lending_club_project/reporting/bad_score_dataset.delta") \
    .saveAsTable("bad_score_dataset")

very_bad_df.write.format("delta") \
    .mode("overwrite") \
    .option("path", "dbfs:/lending_club_project/reporting/very_bad_score_dataset.delta") \
    .saveAsTable("very_bad_score_dataset")